<div style='background-image: url("./seissol2.jpg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 130px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
                <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">TPV5</div>
                <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">SCEC benchmark for a vertical strike-slip fault </div>
        </div>
    </div>

</div>

---

This notebook is based on the benchmakr of TPV 5 of SCEC dynamic code validation project [SCEC](http://scecdata.usc.edu/cvws/).

More infomation about SeisSol can found at [@SeisSol GitHub](https://github.com/SeisSol)

##### Authors:
* Duo Li ([@dli](https://github.com/daisy20170101))
* Thomas Ulrich ([@tulrich](https://github.com/Thomas-Ulrich))
* Elizabeth Madden ([@elizabethmadden](betsymadden@gmail.com))
* Alice Gabriel([@agabriel](https://www.geophysik.uni-muenchende/Members/gabriel))

# Problem discription

## TPV 5

TPV5 is the first SCEC benchmark. It has spontaneous rupture on a vertical strike-slip fault in a homogeneous halfspace. There are slightly heterogeneous initial stress conditions. The earthquake rupture is artificially nucleated in a square zone at the center of the fault surface. The rupture then spontaneously propagates over the rest of the fault surface. As it propagates away from the nucleation zone, it encounters two square patches with initial stress conditions that are different from the rest of the fault surface. 

<img src="TPV5/tpv5_mesh.png"
     alt="Diagram of TPV5. The central square patch is the nucleation zone, while pink and green patches with higher and lower initial stress than neighbour region, respectively."
     style="float: left; margin-right: 10px;height:300px;" />
     



# Fault geometry

The fault within the three-dimensional medium is a vertical right-lateral strike-slip planar fault that resides in a half-space. The fault reaches the Earth’s surface. The rupture is allowed within a rectangular area that is 30000 m long × 15000 m deep. The bottom boundary of and the right and left ends of the allowed 30000 m × 15000 m rupture area are defined by a strength barrier. The nucleation point is centered both along-dip and along-strike of the 30000m × 15000m rupture area, on the fault plane, at 15000m along-strike and 7500m depth.



### Mesh generation


#### GMSH - a free mesh generator

The .geo is the GMSH file for the earthquake model mesh and can be editted directly to change the structural model or the mesh resolution.
The .geo file is then optimized and converted to .neu format.
PUMGen is used to export the mesh into an efficient xdmf format that is used by SeisSol.

More GMSH information can be found at [SeisSol: Gmsh](https://seissol.readthedocs.io/en/latest/gmsh.html)
More PUMGen information can be found at [SeisSol: Meshing with PUMGen](https://seissol.readthedocs.io/en/latest/meshing-with-pumgen.html)


<img src="mesh5.png"
     alt="Image of mesh structure defined in the .geo file"
     style="float: left; margin-right: 10px;height:300px;" />
     




# SeisSol Initialization 


### Launch SeisSol in MPI Cluster

Prerequisite module needs:

- Python (>= 3.6)
- Numpy (>= 1.12.0)
- SCons (>= 3.0, for instructions see below)
- hdf5 (>= 1.8, for instructions see below)
- netcdf (C-Release) (>= 4.4, for instructions see below)
- Intel compiler (>= 17.0, icc, icpc, ifort) or GCC (>= 5.0, gcc, g++, gfortran)
- Some MPI implementation (e.g. OpenMPI)
- ParMETIS for partitioning
- libxsmm (libxsmm_gemm_generator) for small matrix multiplications
- PSpaMM (pspamm.py) for small sparse matrix multiplications (required only on Knights Landing or Skylake)
- CMake (for compiling submodules ImpalaJIT and yaml-cpp)


## Get SeisSol

    git clone https://github.com/SeisSol/SeisSol.git
    git submodule update --init
    


## SeisSol initialization in Argo

    # User specific aliases and functions
    export PATH=$PATH:/home/dli/bin::/home/dli/software/bin
    export LIBRARY_PATH=/home/dli/software/lib:$LIBRARY_PATH
    export LD_LIBRARY_PATH=/home/dli/software/lib:$LD_LIBRARY_PATH

    # module for SeisSol
    module load intel-mkl/2017
    module load intel/2017
    module load openmpi/1.10.2/intel/2017
    module load python/3.6.4
    module load cmake/3.9.2

    # compiler for SeisSol in openmpi/1.10.2/intel/2017
    export CC=mpicc
    export FC=mpif90
    export CXX=mpicxx


## Compilation specification

    # build options
    compileMode = 'release'
    # generatedKernels = 'yes'
    arch = 'dsnb' # use 'dsnb' for SuperMUC phase 1 or use 'dhsw' for SuperMUC phase 2
    parallelization = 'hybrid'
    # measureNodeLevelPerformance = 'none'
    asagi = 'no' # if using Asagi put 'yes', otherwise no.
    commThread = 'no' # Put a 'yes' on phase 2 and 'no' on phase 1 in supermuc
    order = '4' # valid values are 'none', '2', '3', '4', '5', '6', '7', and '8'.

    cppCompiler = 'mpicc'
    fortranCompiler ='mpif90'

    compiler = 'intel'
    equations = 'elastic' # valid values are 'elastic', 'viscoelastic', 'viscoelastic2'
    plasticity = 'yes'
    useExecutionEnvironment = 'yes'
    logLevel = 'warning'
    logLevel0 = 'info'
    netcdf = 'yes'
    hdf5 = 'yes'

    netcdfDir='/home/dli/software'
    hdf5Dir='/home/dli/software'

    metis = 'yes'
    metisDir = '/home/dli/software'
    unitTests = 'none'
    
    # asagi = 'no' ! need to install ASAGI to invoke
    # zlibDir='/home/dli/software/ASAGI/lib/'

 ## Sbath Submission in Argo
 
    #!/bin/bash
    #SBATCH --job-name="Seissol"
    #SBATCH -o job.%j.out
    #SBATCH -e job.%j.err
    #SBATCH -p testing
    #SBATCH -t 0-6:00
    #SBATCH -N 2

    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=8
    #SBATCH --mail-user=d.li@lmu.de
    #SBATCH --mail-type=ALL
    module purge
    module load intel-mkl/2017
    module load intel/2017
    module load openmpi/1.10.2/intel/2017

    export pwd='/home/dli/SeisSol_Cookbook/tpv5'
    #export MP_SINGLE_THREAD=yes
    export OMP_NUM_THREADS=8

    mpirun  ./SeisSol_release_generatedKernels_dsnb_hybrid_none_9_4 parameters.par

### Parameter file

 The .par (an Ascii text file) constains the necessary parameters for SeisSol to run the earthquake model.


#### Easi

SeisSol employs easi files (.yaml) for assigning properties related to fault friction, the stress field and the surrounding material.
These are explained below for this earthquake model. 

More examples can be found at [SeisSol:easi](https://seissol.readthedocs.io/en/latest/easi.html)




### Velocity structure

A materials .yaml contains the constant values assigned in this test case to density (rho), Lame's parameter (lambda) and the shear modulus (mu).



### Fault friction

SeisSol provides a number of friction constitutive laws, e.g. linear slip weakening and rate-and-state. 

This test case utilizes linear slip weakening and this law (30) is assigned in the parameter file.
The frictional parameters (the static and dynamic friction coefficients, mu_s and mu_d, and the slip-weakening distance, d_c) are assigned in the stress .yaml file.
The on-fault cohesion is constant in Scenario B, but increases to 8 MPa above 15 km depth in Scenario A.



### Initial stress condition

SeisSol uses stress tensor ($\sigma_{xx},\sigma_{yy},\sigma_{zz},\sigma_{xy},\sigma_{yz},\sigma_{xz}$) or traction components (normal & shear stress) for initializing background stress condition. 

This test case has a simple, depth-dependent stress field specified in AscSimple_initial_stress.yaml.
The pore fluid pressure also is specified in the stress .yaml and is near-lithostatic.




## Result Visualization

SeisSol provides results in vtk format that is used in [ParaView](https://www.paraview.org/) directly. 


<img src="TPV5/tpv5_SRs_3s.png"
     alt="Image of fault slip rate in ParaView"
     style="float: left; margin-right: 10px;height:300px;" />
     

